
"""
More Variables :

Total Races from current race
- Races in the last 7/14/28/60/90/180/365/730 days 
- Races won in the 7/14/28/60/90/180/365/730 days
- Races placed in the 7/14/28/60/90/180/365/730 days


"""


# Import the code 


In [1]:

#fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one/code/Users/karan.bhatti'
fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti/Dataprep'

import sys
print(sys.path)
print()
#sys.path.append(f'{fileloc}')
print()
#print(sys.path)


['/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti/Code', '/anaconda/envs/azureml_py38/lib/python38.zip', '/anaconda/envs/azureml_py38/lib/python3.8', '/anaconda/envs/azureml_py38/lib/python3.8/lib-dynload', '', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/_project/vendor', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/automl/core/_vendor']




In [2]:

class Data(object):
    pass

df = Data
summ = Data


In [3]:
import helpdesk as hd

In [4]:

import pandas as pd, numpy as np
from datetime import datetime


In [5]:
DEBUG = True

In [6]:

pd.set_option('display.max_rows', 50);
pd.set_option('display.max_columns', 50);
pd.set_option('display.width', 200);


In [7]:

df.ft_raw = pd.read_csv(f'{fileloc}/raw_base_2023-06-05.csv', parse_dates = True)

print(df.ft_raw.shape)
print(df.ft_raw.columns.values.tolist())


(725080, 33)
['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'y_runtime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp', 'x_num_dogs', 'position', 'y_margin', 'y_pos']


### A) Create Variable for Race Profile

In [8]:

df_base = df.ft_raw.copy()


In [9]:
df_base.sort_values(by = ['@id_DOG', 'DogName', 'Event_Dt'], inplace = True)

In [10]:

df_base[df_base.DogName.isin(['FOURTH DIMENTION'])].head()


,@id_DOG,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,PIR,Checks,Comments,SplitMargin,y_runtime,Prizemoney,RaceId,TrainerId,TrainerName,@id_RACE,RaceNum,RaceName,RaceTime,Distance,RaceGrade,Track,date,Event_Dt,ft_sp,x_num_dogs,position,y_margin,y_pos
70,114244064,2,FOURTH DIMENTION,3.0,3,33.4,$4.20,NaN,0.39,0.39,M/22,1,NaN,8.36,22.75,NaN,631783547,131578,P Pollutro,631783547,7,TAB - LONG MAY WE PLAY,05:15PM,400m,Grade 5 T3,Warragul,14 Jan 21,2021-01-14,4.2,8,2,0.39,2
71,114244064,2,FOURTH DIMENTION,8.0,8,33.2,$2.70F,NaN,0.60,0.60,M/33,0,NaN,8.56,23.16,NaN,636866192,131578,P Pollutro,636866192,12,SENNACHIE AT STUD,06:08PM,400m,Grade 5,Warragul,01 Feb 21,2021-02-01,2.7,7,2,0.60,2
72,114244064,8,FOURTH DIMENTION,3.0,3,33.6,$29.60,NaN,7.60,0.36,M/4,1,NaN,4.00,17.61,NaN,645823097,131578,P Pollutro,645823097,12,DOWNLOAD THE NEW WATCHDOG APP,02:35PM,300m,Grade 5,Healesville,02 Mar 21,2021-03-02,29.6,8,8,7.60,8
73,114244064,3,FOURTH DIMENTION,6.0,6,34.0,$9.20,NaN,2.89,0.46,M/66,5,NaN,8.49,23.13,NaN,649065546,131578,P Pollutro,649065546,11,PAUA KENNELS (250+ RANK),06:27PM,400m,Grade 5,Warragul,11 Mar 21,2021-03-11,9.2,7,3,2.89,3
74,114244064,3,FOURTH DIMENTION,4.0,4,33.9,$35.00,NaN,2.09,1.81,M/43,4,NaN,8.96,23.16,NaN,651668743,131578,P Pollutro,651668743,12,TAB - LONG MAY WE PLAY,09:22PM,400m,Grade 5,Warragul,20 Mar 21,2021-03-20,35.0,8,3,2.09,3


In [11]:

# Creating Grade level variables 

"""
df_base['G5'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Grade 5' in x) | ('Five' in x) else 0 )
df_base['G4'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Grade 4' in x) else 0 )
df_base['G3'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Grade 3' in x) else 0 )
df_base['G2'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Grade 2' in x) else 0 )
df_base['G1'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Grade 1' in x) else 0 )

df_base['Gmix'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Mixed' in x) else 0 )
df_base['Gmas'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Master' in x) else 0 )
df_base['Gmai'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Maiden' in x) else 0 )
df_base['Gh'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Heat' in x) else 0 )
df_base['Gres'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Restricted' in x) else 0 )
df_base['Gnov'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Novice' in x) else 0 )
df_base['Gsp'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Special' in x) | ('S/E' in x) else 0 )
df_base['Gfin'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Final' in x) else 0 )
"""


"\ndf_base['G5'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Grade 5' in x) | ('Five' in x) else 0 )\ndf_base['G4'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Grade 4' in x) else 0 )\ndf_base['G3'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Grade 3' in x) else 0 )\ndf_base['G2'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Grade 2' in x) else 0 )\ndf_base['G1'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Grade 1' in x) else 0 )\n\ndf_base['Gmix'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Mixed' in x) else 0 )\ndf_base['Gmas'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Master' in x) else 0 )\ndf_base['Gmai'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Maiden' in x) else 0 )\ndf_base['Gh'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Heat' in x) else 0 )\ndf_base['Gres'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Restricted' in x) else 0 )\ndf_base['Gnov'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Novice' in x) else 0 )\ndf_base['Gsp'] = df_base['Ra

In [12]:

print(datetime.now())

def check_grade(x):
    return pd.Series({
        'G5': int('Grade 5' in x or 'Five' in x),
        'G4': int('Grade 4' in x),
        'G3': int('Grade 3' in x),
        'G2': int('Grade 2' in x),
        'G1': int('Grade 1' in x),
        'Gmix': int('Mixed' in x),
        'Gmas': int('Master' in x),
        'Gmai': int('Maiden' in x),
        'Gh': int('Heat' in x),
        'Gres': int('Restricted' in x),
        'Gnov': int('Novice' in x),
        'Gsp': int('Special' in x or 'S/E' in x),
        'Gfin': int('Final' in x),
    })

df_base = df_base.join(df_base['RaceGrade'].apply(check_grade))

print(datetime.now())


In [13]:

"""
df_base['RaceGrade_H1'] = df_base['RaceGrade'].apply( lambda x : 'Grade 7' if 'Grade 7' in x \
                                                        else 'Grade 6' if 'Grade 6' in x \
                                                        else 'Grade 5' if 'Grade 5' in x \
                                                        else 'Grade 4' if 'Grade 4' in x \
                                                        else 'Grade 3' if 'Grade 3' in x \
                                                        else 'Grade 2' if 'Grade 2' in x \
                                                        else 'Grade 2' if 'Grade 2' in x \
                                                        else 'Grade 1' if 'Grade 1' in x \
                                                        else 'Mixed' if 'Mixed' in x \
                                                        else 'Master' if 'Master' in x \
                                                        else 'Heat' if 'Heat' in x \
                                                        else 'Restricted' if 'Restricted' in x \
                                                        else 'Special' if 'Special' in x \
                                                        else 'Other' )
"""


"\ndf_base['RaceGrade_H1'] = df_base['RaceGrade'].apply( lambda x : 'Grade 7' if 'Grade 7' in x                                                         else 'Grade 6' if 'Grade 6' in x                                                         else 'Grade 5' if 'Grade 5' in x                                                         else 'Grade 4' if 'Grade 4' in x                                                         else 'Grade 3' if 'Grade 3' in x                                                         else 'Grade 2' if 'Grade 2' in x                                                         else 'Grade 2' if 'Grade 2' in x                                                         else 'Grade 1' if 'Grade 1' in x                                                         else 'Mixed' if 'Mixed' in x                                                         else 'Master' if 'Master' in x                                                         else 'Heat' if 'Heat' in x                           

In [15]:

print(datetime.now())

# Optimised code for racegrades
grades = ['Grade 7', 'Grade 6', 'Grade 5', 'Grade 4', 'Grade 3', 'Grade 2', 'Grade 1', 'Mixed', 'Master', 'Heat', 'Restricted', 'Special']

def find_grade(x):
    for grade in grades:
        if grade in x:
            return grade
    return 'Other'

df_base['RaceGrade_H1'] = df_base['RaceGrade'].apply(find_grade)

print(datetime.now())
# <1min runtime


2023-06-07 12:07:46.366314
2023-06-07 12:07:46.597222


In [16]:

df_base['RaceGrade_H1'].value_counts()


Grade 5       287406
Other         164520
Mixed         126510
Grade 6        38822
Heat           35979
Restricted     32439
Grade 7        15091
Grade 4        10630
Special         5519
Grade 1         3361
Grade 3         2154
Master          1812
Grade 2          837
Name: RaceGrade_H1, dtype: int64

In [17]:

grade_fix = df_base.groupby(['RaceGrade_H1','RaceGrade']).agg({'@id_RACE':'nunique'}).reset_index()

grade_fix.to_csv(f'{fileloc}/grade_fix.csv', index = False)


In [18]:

print(df_base.shape)
print(df_base.columns.values.tolist())


(725080, 47)
['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'y_runtime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp', 'x_num_dogs', 'position', 'y_margin', 'y_pos', 'G5', 'G4', 'G3', 'G2', 'G1', 'Gmix', 'Gmas', 'Gmai', 'Gh', 'Gres', 'Gnov', 'Gsp', 'Gfin', 'RaceGrade_H1']


In [20]:

print(pd.DataFrame(df_base.isna().sum()).transpose())


   @id_DOG  Place  DogName  Box  Rug  Weight  StartPrice  Handicap  Margin1  Margin2    PIR  Checks  Comments  SplitMargin  y_runtime  Prizemoney  RaceId  TrainerId  TrainerName  @id_RACE  RaceNum  \
0        0      0        0    0    0       1           0    725080        0   100612  97952       0    351207       119772          0      226219       0          0            0         0        0   

   RaceName  RaceTime  Distance  RaceGrade  Track  date  Event_Dt  ft_sp  x_num_dogs  position  y_margin  y_pos  G5  G4  G3  G2  G1  Gmix  Gmas  Gmai  Gh  Gres  Gnov  Gsp  Gfin  RaceGrade_H1  
0        15         0         0          0      0     0         0      0           0         0         0      0   0   0   0   0   0     0     0     0   0     0     0    0     0             0  


In [21]:

#print(df_base1.RaceGrade.value_counts())
#racegrades = df_base1.RaceGrade.value_counts()
#racegrades.to_csv
#racegrades.to_csv(f"{fileloc}/racegrades.csv", index = True)


In [22]:

#keep_cols_for_export = ['@id_DOG', 'DogName', '@id_RACE','RaceName','Track','RaceGrade','Distance','TrainerName','RunTime','Place','x_races_tot']

# 'TrainerName', 'Track' have been dummified
#cols_to_keep = ['@id_DOG', 'DogName', 'y_runtime', 'RaceId', '@id_RACE', 'position', 'ft_sp', 'x_num_dogs', 'Top3_actual', 'position_random', 'position_fasttrack', 'Winner_fastrack', 'Top3_fastrack', 'Winner_actual', 'Winner_random', 'Top3_random']

cols_to_keep = ['@id_DOG', '@id_RACE','G5', 'G4', 'G3', 'G2', 'G1', 'Gmix', 'Gmas', 'Gmai', 'Gh', 'Gres', 'Gnov', 'Gsp', 'Gfin', 'RaceGrade_H1']

"""
cols_for_drop = ['Place', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin' \
                 , 'Prizemoney', 'TrainerId', 'TrainerName', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date' \
                 , 'Event_Dt','RunTime','ft_sp','position','DogName','RaceId','x_num_dogs']
"""

todaydt = datetime.now().date()
exp_fname = f'dataprep_raceprofile_{todaydt}.csv'

#exp_ds = df_base.drop(columns = cols_for_drop)
exp_ds = df_base[ cols_to_keep ]


In [23]:

print(exp_ds.columns.values.tolist())


['@id_DOG', '@id_RACE', 'G5', 'G4', 'G3', 'G2', 'G1', 'Gmix', 'Gmas', 'Gmai', 'Gh', 'Gres', 'Gnov', 'Gsp', 'Gfin', 'RaceGrade_H1']


In [24]:

exp_ds.head()


,@id_DOG,@id_RACE,G5,G4,G3,G2,G1,Gmix,Gmas,Gmai,Gh,Gres,Gnov,Gsp,Gfin,RaceGrade_H1
0,108390981,626271182,0,0,0,0,0,1,0,0,0,0,0,0,0,Mixed
1,108390981,629797242,1,0,0,0,0,0,0,0,0,0,0,0,0,Grade 5
2,108390981,629797513,1,0,0,0,0,0,0,0,0,0,0,0,0,Grade 5
3,108390981,638810808,0,0,0,0,0,1,0,0,1,0,0,0,0,Mixed
4,108390981,643554706,0,1,0,0,0,0,0,0,0,0,0,0,0,Grade 4


In [25]:
exp_ds.describe()

,@id_DOG,@id_RACE,G5,G4,G3,G2,G1,Gmix,Gmas,Gmai,Gh,Gres,Gnov,Gsp,Gfin
count,7.250800e+05,7.250800e+05,725080.000000,725080.00000,725080.000000,725080.000000,725080.000000,725080.000000,725080.000000,725080.000000,725080.000000,725080.000000,725080.000000,725080.000000,725080.000000
mean,4.866356e+08,7.640069e+08,0.403918,0.01466,0.002971,0.001154,0.004635,0.174477,0.021930,0.153033,0.103917,0.052618,0.038023,0.032457,0.040462
std,1.219313e+08,7.786416e+07,0.490682,0.12019,0.054423,0.033956,0.067925,0.379520,0.146455,0.360019,0.305153,0.223269,0.191253,0.177211,0.197040
min,1.083910e+08,6.238619e+08,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.159944e+08,6.985139e+08,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.791464e+08,7.654781e+08,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.620734e+08,8.292900e+08,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.145360e+09,8.978632e+08,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [26]:

print(pd.DataFrame(exp_ds.isna().sum()).transpose())


   @id_DOG  @id_RACE  G5  G4  G3  G2  G1  Gmix  Gmas  Gmai  Gh  Gres  Gnov  Gsp  Gfin  RaceGrade_H1
0        0         0   0   0   0   0   0     0     0     0   0     0     0    0     0             0


In [27]:

print(datetime.now())
exp_ds.to_csv(f'{fileloc}/{exp_fname}', index = False)
print(datetime.now())


2023-06-07 12:15:26.807316
2023-06-07 12:15:34.520290


In [28]:

print('THE END')


THE END
